In [164]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu, True)

In [165]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import MinMaxScaler
import random

In [166]:
tf.keras.backend.clear_session()

In [167]:
def add_cos_monthly(df, column_name):
    values = [np.cos(2 * np.pi * (i % 12) / 12) for i in range(len(df))]
    df[column_name] = values
    return df

In [168]:
subject = 'Tocantins - Consumo de Cimento (t)'
start_index = 0
split_index = 191 #Referente aos 230 anos de input 
window_size = 36
train_split = split_index + 1 - 30

In [169]:
data = pd.read_csv('2003_mo_model_input_TO.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data =data.drop([' NFSP - Fluxo Mensal (Milhões de reais)'], axis=1)
data

,Unnamed: 0,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Tocantins - Desemprego,Tocantins - PIB - Estadual,Tocantins - PIB - Construção Civil,Tocantins - PIB - Per Capita,Tocantins - PIB - Preços de Mercado,Tocantins - Produção de Cimento (t),Tocantins - IDH,Tocantins - Consumo de Cimento (t)
0,2003-1,0.724032,11.520143,1.639718,1.036534,1.772069e+08,7.330309e+06,0.969649,8.243830,1.593908e+07,1.457380e+06,10.276585,2.572692e+07,12.733000,0.700107,17.555
1,2003-2,0.690297,11.189862,1.378899,0.993449,1.773884e+08,7.335910e+06,0.950783,8.237931,1.596108e+07,1.459029e+06,10.281590,2.576177e+07,12.733000,0.700487,14.976
2,2003-3,0.669681,10.820792,1.924317,0.973020,1.775699e+08,7.341511e+06,0.938332,8.232033,1.598308e+07,1.460678e+06,10.286595,2.579662e+07,12.733000,0.700866,13.334
3,2003-4,0.660494,10.417840,1.331174,0.940489,1.777514e+08,7.347112e+06,0.926401,8.226135,1.600508e+07,1.462328e+06,10.291600,2.583147e+07,12.733000,0.701246,15.173
4,2003-5,0.648337,9.959690,1.736072,0.917493,1.779329e+08,7.352713e+06,0.951683,8.220236,1.602709e+07,1.463977e+06,10.296605,2.586632e+07,12.733000,0.701626,18.728
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,2022-8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,67.097178,NaN,54.154
236,2022-9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,67.077411,NaN,55.664
237,2022-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,66.563807,NaN,52.320
238,2022-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,65.682099,NaN,43.036


In [170]:
input_data = data.iloc[:split_index + 1,1:-1]
# mean = np.mean(input_data, axis=0)
# stddev =  np.std(input_data, axis=0)
# input_data = ((input_data - mean) /stddev)
scaler=MinMaxScaler()
scaled_data = scaler.fit_transform(input_data)
input_data = pd.DataFrame(scaled_data, columns=input_data.columns)
input_data

,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Tocantins - Desemprego,Tocantins - PIB - Estadual,Tocantins - PIB - Construção Civil,Tocantins - PIB - Per Capita,Tocantins - PIB - Preços de Mercado,Tocantins - Produção de Cimento (t),Tocantins - IDH
0,1.000000,1.000000,0.746035,1.000000,0.000000,0.000000,1.000000,0.099694,0.000000,0.670765,0.000000,0.537233,2.775558e-16,0.000000
1,0.921240,0.965719,0.580259,0.936603,0.007992,0.011007,0.968938,0.097341,0.005442,0.675607,0.010211,0.543587,2.775558e-16,0.010615
2,0.873107,0.927412,0.926926,0.906543,0.015984,0.022015,0.948438,0.094989,0.010884,0.680448,0.020423,0.549940,2.775558e-16,0.021231
3,0.851659,0.885588,0.549924,0.858675,0.023976,0.033022,0.928794,0.092636,0.016327,0.685289,0.030634,0.556293,2.775558e-16,0.031846
4,0.823277,0.838034,0.807277,0.824839,0.031968,0.044030,0.970421,0.090284,0.021769,0.690130,0.040846,0.562647,2.775558e-16,0.042462
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,0.000000,0.030808,0.063780,0.381401,0.987600,0.828698,0.005731,0.671505,0.983358,0.150711,0.994888,0.237288,9.734994e-01,0.969542
188,0.029504,0.027720,0.048101,0.466609,0.990700,0.823090,0.037761,0.666733,0.981440,0.153335,0.994158,0.240931,9.816994e-01,0.966685
189,0.043125,0.023074,0.016800,0.530936,0.993800,0.817482,0.087044,0.661961,0.979521,0.155958,0.993427,0.244574,9.950540e-01,0.963829
190,0.059756,0.017547,0.035186,0.611274,0.996900,0.811875,0.118264,0.657189,0.977602,0.158582,0.992697,0.248216,9.930917e-01,0.960972


In [171]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-12)
target_data

0      19.115
1      22.973
2      30.126
3      29.497
4      29.528
        ...  
235       NaN
236       NaN
237       NaN
238       NaN
239       NaN
Name: Tocantins - Consumo de Cimento (t), Length: 240, dtype: float64

In [172]:
# input para treinamento
train_input = input_data.iloc[start_index:train_split]
train_input

,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Tocantins - Desemprego,Tocantins - PIB - Estadual,Tocantins - PIB - Construção Civil,Tocantins - PIB - Per Capita,Tocantins - PIB - Preços de Mercado,Tocantins - Produção de Cimento (t),Tocantins - IDH
0,1.000000,1.000000,0.746035,1.000000,0.000000,0.000000,1.000000,0.099694,0.000000,0.670765,0.000000,0.537233,2.775558e-16,0.000000
1,0.921240,0.965719,0.580259,0.936603,0.007992,0.011007,0.968938,0.097341,0.005442,0.675607,0.010211,0.543587,2.775558e-16,0.010615
2,0.873107,0.927412,0.926926,0.906543,0.015984,0.022015,0.948438,0.094989,0.010884,0.680448,0.020423,0.549940,2.775558e-16,0.021231
3,0.851659,0.885588,0.549924,0.858675,0.023976,0.033022,0.928794,0.092636,0.016327,0.685289,0.030634,0.556293,2.775558e-16,0.031846
4,0.823277,0.838034,0.807277,0.824839,0.031968,0.044030,0.970421,0.090284,0.021769,0.690130,0.040846,0.562647,2.775558e-16,0.042462
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,0.379453,0.024030,0.338208,0.039971,0.890665,0.956993,0.182507,0.940224,0.979603,0.056573,0.961460,0.130871,7.930153e-01,0.995979
158,0.332832,0.021531,0.371581,0.027548,0.893949,0.953662,0.168812,0.923662,0.981457,0.060333,0.964540,0.134501,7.947397e-01,0.996344
159,0.313954,0.019385,0.365349,0.037038,0.897232,0.950331,0.152959,0.907101,0.983311,0.064092,0.967620,0.138131,7.965109e-01,0.996710
160,0.294856,0.017671,0.404993,0.070841,0.900516,0.947000,0.137613,0.890540,0.985165,0.067852,0.970700,0.141761,7.994809e-01,0.997076


In [173]:
# Alvo para treinamento
train_target = target_data.iloc[start_index:train_split]
train_target

0      19.115
1      22.973
2      30.126
3      29.497
4      29.528
        ...  
157    33.787
158    41.594
159    36.019
160    45.143
161    41.704
Name: Tocantins - Consumo de Cimento (t), Length: 162, dtype: float64

In [174]:
def create_batches(t_input, t_target, window_size, start_from):
    
    X_batches = []
    y_batches = []

    train_input_values = t_input.values 

    for i in range(len(t_input) - window_size):
        
        X_window = train_input_values[i:i+window_size, :]
        y_target = t_target[start_from+i+window_size]

        X_batches.append(X_window)
        y_batches.append(y_target)

    return np.array(X_batches), np.array(y_batches)

In [175]:
reshaped_train, reshaped_target = create_batches(train_input, 
                                                 train_target, 
                                                 window_size, 
                                                 start_index)
reshaped_train.shape

(126, 36, 14)

In [176]:
# input para treinamento
test_input = input_data.iloc[train_split - window_size:split_index + 1]
test_input

,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Tocantins - Desemprego,Tocantins - PIB - Estadual,Tocantins - PIB - Construção Civil,Tocantins - PIB - Per Capita,Tocantins - PIB - Preços de Mercado,Tocantins - Produção de Cimento (t),Tocantins - IDH
126,0.612440,0.052718,0.094523,0.201912,0.780557,0.994213,0.372054,0.803309,0.829822,0.008257,0.839725,0.021269,0.816609,0.724393
127,0.626227,0.056653,0.200555,0.209545,0.784414,0.995177,0.375438,0.815610,0.836445,0.009633,0.844663,0.024814,0.814905,0.725702
128,0.645442,0.060771,0.214993,0.238038,0.788271,0.996142,0.372254,0.827911,0.843068,0.011009,0.849602,0.028359,0.829914,0.727010
129,0.673151,0.065105,0.053026,0.249843,0.792128,0.997106,0.369458,0.840212,0.849691,0.012385,0.854541,0.031904,0.835037,0.728319
130,0.699949,0.069099,0.016760,0.266770,0.795984,0.998071,0.371547,0.852513,0.856314,0.013761,0.859479,0.035449,0.834371,0.729628
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,0.000000,0.030808,0.063780,0.381401,0.987600,0.828698,0.005731,0.671505,0.983358,0.150711,0.994888,0.237288,0.973499,0.969542
188,0.029504,0.027720,0.048101,0.466609,0.990700,0.823090,0.037761,0.666733,0.981440,0.153335,0.994158,0.240931,0.981699,0.966685
189,0.043125,0.023074,0.016800,0.530936,0.993800,0.817482,0.087044,0.661961,0.979521,0.155958,0.993427,0.244574,0.995054,0.963829
190,0.059756,0.017547,0.035186,0.611274,0.996900,0.811875,0.118264,0.657189,0.977602,0.158582,0.992697,0.248216,0.993092,0.960972


In [177]:
reshaped_test, reshaped_test_target = create_batches(test_input, 
                                                     target_data, 
                                                     window_size, 
                                                     train_split - window_size)
reshaped_test.shape 

(30, 36, 14)

In [178]:
def validation_splitter(arr, div_factor, add_factor=0):
    split_factor = len(arr) // div_factor
    positions_to_drop_index = []
    positions_to_drop = []
    for i in range(split_factor):
        pos = len(arr) - (i * div_factor + 1)
        positions_to_drop_index.append(pos)
        positions_to_drop.append(pos + add_factor)
    
    arr_droped = arr[positions_to_drop]
    arr_result = np.delete(arr, positions_to_drop_index, axis=0)
    
    return arr_result, arr_droped

In [193]:
# Rede Neural Recorrente com optmizador Estocástico
def bidirectional_lstm_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    train, train_val = validation_splitter(train_input, 7)
    target,target_val = validation_splitter(train_target, 7)
#     display(train.shape)
#     display(train_val.shape)
#     display(target.shape)
#     display(target_val.shape)
    # Método estocástico e learning rate=0.005
#     optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.Bidirectional(
            tf.keras.layers.LSTM(36, activation='tanh', 
                                 return_sequences=False), 
                                 input_shape=(train_input.shape[1], 
                                              train_input.shape[2])),
#         tf.keras.layers.Dropout(0.5),
#         tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(180, activation='tanh')),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.005), loss='mean_squared_error')   
    history = model.fit(train, 
                        target, 
                        epochs=10000,
                        validation_data=(train_val,
                                         target_val),
                        callbacks=[early_stopping], 
                        verbose=want_verbose)
    return model, history

In [194]:
def get_a_good_seed(train_input, train_target, test_input, test_target):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(5)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = bidirectional_lstm_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")
            if winner_seed == 0.0:
                return winner_seed

    return winner_seed

In [195]:
winner_seed = get_a_good_seed(reshaped_train, reshaped_target, reshaped_test, reshaped_test_target)
print(f"\n\nfinal_seed: {winner_seed}")

[2971600647, 3066300271, 2236801475, 3804779778, 97524878]


Step: 0 ___________________________________________
val_loss: 9.163615226745605
winner_seed: 2971600647


Step: 1 ___________________________________________
val_loss: 10.075328826904297


Step: 2 ___________________________________________
val_loss: 10.175830841064453


Step: 3 ___________________________________________
val_loss: 8.591015815734863
winner_seed: 3804779778


Step: 4 ___________________________________________
val_loss: 10.53941535949707


final_seed: 3804779778


In [196]:
trained_model, history = bidirectional_lstm_model(reshaped_train, 
                                                  reshaped_target, 
                                                  want_verbose=1, 
                                                  seed=winner_seed)

Epoch 1/10000
4/4 [==============================] - 3s 135ms/step - loss: 1736.4913 - val_loss: 1558.8966
Epoch 2/10000
4/4 [==============================] - 0s 10ms/step - loss: 1483.5618 - val_loss: 1315.1968
Epoch 3/10000
4/4 [==============================] - 0s 10ms/step - loss: 1258.6924 - val_loss: 1119.6919
Epoch 4/10000
4/4 [==============================] - 0s 10ms/step - loss: 1067.6730 - val_loss: 944.0497
Epoch 5/10000
4/4 [==============================] - 0s 10ms/step - loss: 902.1309 - val_loss: 802.9226
Epoch 6/10000
4/4 [==============================] - 0s 12ms/step - loss: 770.9241 - val_loss: 693.0417
Epoch 7/10000
4/4 [==============================] - 0s 10ms/step - loss: 667.7468 - val_loss: 602.9794
Epoch 8/10000
4/4 [==============================] - 0s 11ms/step - loss: 581.1691 - val_loss: 526.1179
Epoch 9/10000
4/4 [==============================] - 0s 13ms/step - loss: 508.3254 - val_loss: 458.6826
Epoch 10/10000
4/4 [==============================] - 0s

4/4 [==============================] - 0s 10ms/step - loss: 48.2120 - val_loss: 42.3749
Epoch 81/10000
4/4 [==============================] - 0s 11ms/step - loss: 48.2011 - val_loss: 42.3531
Epoch 82/10000
4/4 [==============================] - 0s 11ms/step - loss: 48.1715 - val_loss: 42.3351
Epoch 83/10000
4/4 [==============================] - 0s 11ms/step - loss: 48.1800 - val_loss: 42.3131
Epoch 84/10000
4/4 [==============================] - 0s 12ms/step - loss: 48.1500 - val_loss: 42.3032
Epoch 85/10000
4/4 [==============================] - 0s 11ms/step - loss: 48.1611 - val_loss: 42.2969
Epoch 86/10000
4/4 [==============================] - 0s 13ms/step - loss: 48.1474 - val_loss: 42.2946
Epoch 87/10000
4/4 [==============================] - 0s 12ms/step - loss: 48.1457 - val_loss: 42.2930
Epoch 88/10000
4/4 [==============================] - 0s 10ms/step - loss: 48.1504 - val_loss: 42.2920
Epoch 89/10000
4/4 [==============================] - 0s 10ms/step - loss: 48.1505 - val

4/4 [==============================] - 0s 12ms/step - loss: 39.5203 - val_loss: 32.7326
Epoch 160/10000
4/4 [==============================] - 0s 10ms/step - loss: 39.1091 - val_loss: 32.2390
Epoch 161/10000
4/4 [==============================] - 0s 11ms/step - loss: 38.6236 - val_loss: 31.9069
Epoch 162/10000
4/4 [==============================] - 0s 11ms/step - loss: 38.2502 - val_loss: 31.5111
Epoch 163/10000
4/4 [==============================] - 0s 11ms/step - loss: 37.8400 - val_loss: 31.1123
Epoch 164/10000
4/4 [==============================] - 0s 10ms/step - loss: 37.5432 - val_loss: 30.7317
Epoch 165/10000
4/4 [==============================] - 0s 11ms/step - loss: 37.1714 - val_loss: 30.4577
Epoch 166/10000
4/4 [==============================] - 0s 11ms/step - loss: 36.8042 - val_loss: 30.0525
Epoch 167/10000
4/4 [==============================] - 0s 10ms/step - loss: 36.4643 - val_loss: 29.5871
Epoch 168/10000
4/4 [==============================] - 0s 11ms/step - loss: 36.1

Epoch 238/10000
4/4 [==============================] - 0s 11ms/step - loss: 29.5228 - val_loss: 23.7792
Epoch 239/10000
4/4 [==============================] - 0s 10ms/step - loss: 29.1169 - val_loss: 23.7340
Epoch 240/10000
4/4 [==============================] - 0s 10ms/step - loss: 29.0135 - val_loss: 23.9286
Epoch 241/10000
4/4 [==============================] - 0s 11ms/step - loss: 28.9547 - val_loss: 24.1992
Epoch 242/10000
4/4 [==============================] - 0s 11ms/step - loss: 29.1376 - val_loss: 24.0545
Epoch 243/10000
4/4 [==============================] - 0s 10ms/step - loss: 28.7832 - val_loss: 23.7052
Epoch 244/10000
4/4 [==============================] - 0s 10ms/step - loss: 28.7788 - val_loss: 23.7426
Epoch 245/10000
4/4 [==============================] - 0s 10ms/step - loss: 29.1848 - val_loss: 24.3110
Epoch 246/10000
4/4 [==============================] - 0s 11ms/step - loss: 28.8476 - val_loss: 23.9941
Epoch 247/10000
4/4 [==============================] - 0s 10ms/s

Epoch 317/10000
4/4 [==============================] - 0s 10ms/step - loss: 24.7443 - val_loss: 26.0006
Epoch 318/10000
4/4 [==============================] - 0s 9ms/step - loss: 24.3990 - val_loss: 25.3115
Epoch 319/10000
4/4 [==============================] - 0s 10ms/step - loss: 24.4277 - val_loss: 27.2636
Epoch 320/10000
4/4 [==============================] - 0s 10ms/step - loss: 23.7034 - val_loss: 27.9724
Epoch 321/10000
4/4 [==============================] - 0s 10ms/step - loss: 24.6728 - val_loss: 27.8129
Epoch 322/10000
4/4 [==============================] - 0s 9ms/step - loss: 23.7321 - val_loss: 26.9610
Epoch 323/10000
4/4 [==============================] - 0s 10ms/step - loss: 23.8247 - val_loss: 25.8983
Epoch 324/10000
4/4 [==============================] - 0s 10ms/step - loss: 23.6624 - val_loss: 24.9332
Epoch 325/10000
4/4 [==============================] - 0s 9ms/step - loss: 23.5438 - val_loss: 25.7403
Epoch 326/10000
4/4 [==============================] - 0s 10ms/step

...

Epoch 713/10000
4/4 [==============================] - 0s 11ms/step - loss: 5.8783 - val_loss: 14.0838
Epoch 714/10000
4/4 [==============================] - 0s 12ms/step - loss: 7.2066 - val_loss: 16.4952
Epoch 715/10000
4/4 [==============================] - 0s 12ms/step - loss: 7.4628 - val_loss: 18.9792
Epoch 716/10000
4/4 [==============================] - 0s 10ms/step - loss: 6.6357 - val_loss: 21.8005
Epoch 717/10000
4/4 [==============================] - 0s 10ms/step - loss: 6.4386 - val_loss: 17.2000
Epoch 718/10000
4/4 [==============================] - 0s 10ms/step - loss: 6.2742 - val_loss: 17.9122
Epoch 719/10000
4/4 [==============================] - 0s 10ms/step - loss: 6.4724 - val_loss: 19.7077
Epoch 720/10000
4/4 [==============================] - 0s 10ms/step - loss: 5.6350 - val_loss: 18.6006
Epoch 721/10000
4/4 [==============================] - 0s 13ms/step - loss: 5.6610 - val_loss: 15.2437
Epoch 722/10000
4/4 [==============================] - 0s 11ms/step - los

4/4 [==============================] - 0s 10ms/step - loss: 5.6627 - val_loss: 12.8916
Epoch 793/10000
4/4 [==============================] - 0s 11ms/step - loss: 5.4543 - val_loss: 19.0886
Epoch 794/10000
4/4 [==============================] - 0s 10ms/step - loss: 5.5849 - val_loss: 22.6451
Epoch 795/10000
4/4 [==============================] - 0s 11ms/step - loss: 5.6390 - val_loss: 15.8963
Epoch 796/10000
4/4 [==============================] - 0s 10ms/step - loss: 5.8493 - val_loss: 14.6167
Epoch 797/10000
4/4 [==============================] - 0s 10ms/step - loss: 6.1773 - val_loss: 13.7209
Epoch 798/10000
4/4 [==============================] - 0s 10ms/step - loss: 5.2484 - val_loss: 16.7676
Epoch 799/10000
4/4 [==============================] - 0s 10ms/step - loss: 5.4811 - val_loss: 21.0473
Epoch 800/10000
4/4 [==============================] - 0s 10ms/step - loss: 4.8891 - val_loss: 22.0917
Epoch 801/10000
4/4 [==============================] - 0s 10ms/step - loss: 5.7216 - val_

4/4 [==============================] - 0s 13ms/step - loss: 4.6905 - val_loss: 16.7670
Epoch 872/10000
4/4 [==============================] - 0s 12ms/step - loss: 4.8328 - val_loss: 16.3730
Epoch 873/10000
4/4 [==============================] - 0s 10ms/step - loss: 4.6474 - val_loss: 18.1971
Epoch 874/10000
4/4 [==============================] - 0s 10ms/step - loss: 4.9271 - val_loss: 15.5643
Epoch 875/10000
4/4 [==============================] - 0s 10ms/step - loss: 4.8416 - val_loss: 12.4566
Epoch 876/10000
4/4 [==============================] - 0s 10ms/step - loss: 6.0512 - val_loss: 25.6179
Epoch 877/10000
4/4 [==============================] - 0s 11ms/step - loss: 7.3288 - val_loss: 29.5182
Epoch 878/10000
4/4 [==============================] - 0s 10ms/step - loss: 12.3129 - val_loss: 25.0130
Epoch 879/10000
4/4 [==============================] - 0s 10ms/step - loss: 22.5212 - val_loss: 22.4780
Epoch 880/10000
4/4 [==============================] - 0s 10ms/step - loss: 17.5944 - v

4/4 [==============================] - 0s 11ms/step - loss: 6.8488 - val_loss: 15.1475
Epoch 951/10000
4/4 [==============================] - 0s 10ms/step - loss: 7.0121 - val_loss: 15.5995
Epoch 952/10000
4/4 [==============================] - 0s 10ms/step - loss: 6.7537 - val_loss: 16.3808
Epoch 953/10000
4/4 [==============================] - 0s 10ms/step - loss: 6.7913 - val_loss: 16.8967
Epoch 954/10000
4/4 [==============================] - 0s 10ms/step - loss: 6.6070 - val_loss: 15.3458
Epoch 955/10000
4/4 [==============================] - 0s 10ms/step - loss: 6.9946 - val_loss: 16.6279
Epoch 956/10000
4/4 [==============================] - 0s 11ms/step - loss: 6.4627 - val_loss: 16.9815
Epoch 957/10000
4/4 [==============================] - 0s 10ms/step - loss: 6.7913 - val_loss: 16.2838
Epoch 958/10000
4/4 [==============================] - 0s 10ms/step - loss: 6.6001 - val_loss: 16.8240
Epoch 959/10000
4/4 [==============================] - 0s 10ms/step - loss: 6.6302 - val_

4/4 [==============================] - 0s 10ms/step - loss: 5.5421 - val_loss: 18.8335
Epoch 1030/10000
4/4 [==============================] - 0s 11ms/step - loss: 5.4584 - val_loss: 18.7899
Epoch 1031/10000
4/4 [==============================] - 0s 11ms/step - loss: 5.3541 - val_loss: 15.9274
Epoch 1032/10000
4/4 [==============================] - 0s 11ms/step - loss: 5.2700 - val_loss: 16.8931
Epoch 1033/10000
4/4 [==============================] - 0s 11ms/step - loss: 5.4603 - val_loss: 17.0371
Epoch 1034/10000
4/4 [==============================] - 0s 9ms/step - loss: 5.1690 - val_loss: 17.5890
Epoch 1035/10000
4/4 [==============================] - 0s 11ms/step - loss: 5.5275 - val_loss: 18.0255
Epoch 1036/10000
4/4 [==============================] - 0s 11ms/step - loss: 5.3787 - val_loss: 18.6000
Epoch 1037/10000
4/4 [==============================] - 0s 11ms/step - loss: 5.4206 - val_loss: 18.0489
Epoch 1038/10000
4/4 [==============================] - 0s 10ms/step - loss: 5.293

In [197]:
def mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    results_data = []

    for i in range(len(test_target)):
        prediction = model.predict(test_input[i:i+1])
        target = test_target[start_target + i]
        error = np.abs(prediction - target)
        errors.append(error)
        error_percent.append(error/target)
        results_data.append([f"Month-{i + 1}", 
                             prediction[0][0], 
                             target, 
                             error[0][0]])

    df_results = pd.DataFrame(results_data, columns=["Month", "Prediction", "Target", "Error"])

    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return df_results, mae, mape

In [198]:
df_results, mae, mape = mae_mape_calculator(trained_model, 
                                            reshaped_test, 
                                            reshaped_test_target, 
                                            start_index)
pd.set_option('display.max_columns', None)
df_results.T

1/1 [==============================] - 0s 17ms/step


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
Month,Month-1,Month-2,Month-3,Month-4,Month-5,Month-6,Month-7,Month-8,Month-9,Month-10,Month-11,Month-12,Month-13,Month-14,Month-15,Month-16,Month-17,Month-18,Month-19,Month-20,Month-21,Month-22,Month-23,Month-24,Month-25,Month-26,Month-27,Month-28,Month-29,Month-30
Prediction,40.66959,40.560581,40.29179,40.100536,39.953968,39.769249,39.774044,40.169827,40.453705,40.523495,40.596569,40.860531,40.549744,40.70332,41.108986,41.126102,40.828831,40.374058,40.668056,41.246578,41.542629,41.735424,41.805599,41.827984,41.823524,41.785946,41.733814,41.78339,42.049603,42.504032
Target,45.71,45.564,43.754,48.554,40.896,35.255,34.018,22.422,29.555,28.919,25.589,40.237,37.012,38.793,38.281,39.711,33.695,28.206,31.615,30.275,29.568,26.94,35.91,33.954,37.983,40.046,32.647,39.065,36.18,28.749
Error,5.040409,5.003418,3.462212,8.453465,0.942032,4.514248,5.756042,17.747826,10.898705,11.604494,15.007568,0.623531,3.537743,1.91032,2.827988,1.415104,7.133831,12.168058,9.053057,10.971579,11.974628,14.795424,5.895599,7.873985,3.840523,1.739944,9.086815,2.718391,5.869602,13.755032


In [199]:
display(mae)
display(mape)

7.187385

0.232893

In [200]:
def year_mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    
    target_sum = 0
    prediction_sum = 0
    
    for i in range(len(test_target)):
        if i % 12 == 0 and i != 0:
            error = np.abs(target_sum - prediction_sum)
            errors.append(error)
            error_percent.append(error / target_sum)
            print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
            target_sum = 0
            prediction_sum = 0
            
        prediction = model.predict(test_input[i:i+1])
        target_sum += test_target[start_target + i]
        prediction_sum += prediction
        
    error = np.abs(target_sum - prediction_sum)
    errors.append(error)
    error_percent.append(error / target_sum)
    print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
        
    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return errors, mae, mape

In [201]:
errors, mae, mape = year_mae_mape_calculator(trained_model, 
                                             reshaped_test, 
                                             reshaped_test_target, 
                                             start_index)
display(errors)
display(mae)
display(mape)

1/1 [==============================] - 0s 17ms/step
Ano-0: |Prediction[[483.72388]] - Target[440.473]| =  Error: [[43.250885]]; MAPE:[[0.09819191]]
1/1 [==============================] - 0s 16ms/step
Ano-0: |Prediction[[493.5173]] - Target[403.96]| =  Error: [[89.55731]]; MAPE:[[0.22169846]]
1/1 [==============================] - 0s 15ms/step
Ano-5: |Prediction[[251.6803]] - Target[214.67]| =  Error: [[37.0103]]; MAPE:[[0.17240556]]


[array([[43.250885]], dtype=float32),
 array([[89.55731]], dtype=float32),
 array([[37.0103]], dtype=float32)]

56.606167

0.16409864